In [48]:


pip install xgboost --upgrade

Note: you may need to restart the kernel to use updated packages.


In [49]:
!pip install pandas xgboost

In [50]:
!pip install xgboost

In [51]:
!pip install --upgrade pandas


In [52]:
import pandas as pd
print(hasattr(pd, 'Int64Index'))

False


In [53]:
import xgboost as xgb

In [54]:
from datetime import datetime 
from typing import Tuple 
import numpy as np 
import pandas as pd

In [55]:
df = pd.read_parquet('/Users/chetanhalai/Documents/code base for projects/4) uber/FINAL TAXI PROJECT 2023/tabular_data_final.parquet')
df.head()

,rides_previous_648_hour,rides_previous_647_hour,rides_previous_646_hour,rides_previous_645_hour,rides_previous_644_hour,rides_previous_643_hour,rides_previous_642_hour,rides_previous_641_hour,rides_previous_640_hour,rides_previous_639_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,19.0,28.0,43.0,33.0,12.0,3.0,2.0,1.0,1.0,1.0,...,3.0,4.0,5.0,3.0,8.0,18.0,22.0,2023-01-28,4,58.0
1,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,2.0,0.0,...,5.0,5.0,7.0,5.0,8.0,15.0,26.0,2023-01-29,4,53.0
2,2.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,2.0,3.0,...,3.0,1.0,5.0,2.0,1.0,2.0,0.0,2023-01-30,4,2.0
3,3.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,6.0,2.0,...,2.0,1.0,1.0,1.0,0.0,0.0,2.0,2023-01-31,4,0.0
4,3.0,16.0,28.0,21.0,12.0,5.0,4.0,2.0,2.0,3.0,...,4.0,2.0,3.0,0.0,5.0,1.0,2.0,2023-01-28,7,1.0


In [56]:
def train_test_split(
    df: pd.DataFrame,
    cutoff_date: datetime,
    target_column_name: str,
    ) -> Tuple[pd.DataFrame, pd.Series, pd.DataFrame, pd.Series]:
    """
    """
    train_data = df[df.pickup_hour < cutoff_date].reset_index(drop=True)
    test_data = df[df.pickup_hour >= cutoff_date].reset_index(drop=True)

    X_train = train_data.drop(columns=[target_column_name])
    y_train = train_data[target_column_name]
    X_test = test_data.drop(columns=[target_column_name])
    y_test = test_data[target_column_name]

    return X_train, y_train, X_test, y_test

In [57]:
X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2023, 4, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}') 

X_train.shape=(2330, 650)
y_train.shape=(2330,)
X_test.shape=(2592, 650)
y_test.shape=(2592,)


In [58]:
# use only past rides data
past_rides_columns = [c for c in X_train.columns if c.startswith('rides_')]
X_train_only_numeric = X_train[past_rides_columns]

In [59]:
#convert to numpy values 
X_train_only_numeric = X_train_only_numeric.values

In [60]:
#convert to numpy values 
y_train = y_train.values

In [61]:
model = xgb.XGBRegressor()
model.fit(X_train_only_numeric, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [62]:
X_test_only_numeric = X_test[past_rides_columns]
predictions = model.predict(X_test_only_numeric)

test_mae = mean_absolute_error(y_test, predictions)
print(test_mae)

AttributeError: module 'pandas' has no attribute 'Int64Index'